# Check
* Above real-time segmentation on high resolution images (1024 x 2048px)
* Yield Accuracy of 68% mean intersection over union
* Process Input at 123.5 frames per second on Cityscapes dataset
* No large pre-training required
* Combines spatial details at High resolution with deep features extracted at lower resolution

In [1]:
import keras

Using TensorFlow backend.


In [2]:
def conv_block(inputs, conv_type, kernel, kernel_size, strides, padding='same', relu=True):
    if(conv_type == 'ds'):
        x = keras.layers.SeparableConv2D(kernel, kernel_size, padding=padding, strides = strides)(inputs)
    else:
        x = keras.layers.Conv2D(kernel, kernel_size, padding=padding, strides = strides)(inputs)  

    x = keras.layers.BatchNormalization()(x)

    if (relu):
        x = keras.layers.Activation('relu')(x)
  
    return x

In [3]:
def _res_bottleneck(inputs, filters, kernel, t, s, r=False):
    
    tchannel = keras.backend.int_shape(inputs)[-1] * t

    x = conv_block(inputs, 'conv', tchannel, (1, 1), strides=(1, 1))

    x = keras.layers.DepthwiseConv2D(kernel, strides=(s, s), depth_multiplier=1, padding='same')(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation("relu")(x)

    x = conv_block(x, 'conv', filters, (1, 1), strides=(1, 1), padding='same', relu=False)

    if r:
        x = keras.layers.add([x, inputs])
    return x

In [4]:
def bottleneck_block(inputs, filters, kernel, t, strides, n):
    x = _res_bottleneck(inputs, filters, kernel, t, strides)

    for i in range(1, n):
        x = _res_bottleneck(x, filters, kernel, t, 1, True)

    return x

In [5]:
def pyramid_pooling_block(input_tensor, bin_sizes):
    concat_list = [input_tensor]
    w = 64
    h = 32

    for bin_size in bin_sizes:
        x = keras.layers.AveragePooling2D(pool_size=(w//bin_size, h//bin_size), strides=(w//bin_size, h//bin_size))(input_tensor)
        x = keras.layers.Conv2D(128, 3, 2)(x)
        x = keras.layers.Lambda(lambda x: keras.backend.image.resize(x, (w,h)))(x)

    concat_list.append(x)

    return keras.layers.concatenate(concat_list)

In [6]:
def create_model():
    input_layer = keras.layers.Input(shape=(2048, 1024, 3), name = 'input_layer')
    
    lds_layer = conv_block(input_layer, 'conv', 32, (3, 3), strides = (2, 2))
    lds_layer = conv_block(lds_layer, 'ds', 48, (3, 3), strides = (2, 2))
    lds_layer = conv_block(lds_layer, 'ds', 64, (3, 3), strides = (2, 2))
    
    gfe_layer = bottleneck_block(lds_layer, 64, (3, 3), t=6, strides=2, n=3)
    gfe_layer = bottleneck_block(gfe_layer, 96, (3, 3), t=6, strides=2, n=3)
    gfe_layer = bottleneck_block(gfe_layer, 128, (3, 3), t=6, strides=1, n=3)
    
    gfe_layer = pyramid_pooling_block(gfe_layer, [2,4,6,8])
    
    ff_layer1 = conv_block(lds_layer, 'conv', 128, (1,1), padding='same', strides= (1,1), relu=False)
    
    ff_layer2 = keras.layers.UpSampling2D((4, 4))(gfe_layer)
    ff_layer2 = tf.keras.layers.SeparableConv2D(128, (3, 3), padding='same', strides = (1, 1), activation=None, dilation_rate=(4, 4))(ff_layer2)
    ff_layer2 = tf.keras.layers.BatchNormalization()(ff_layer2)
    
    ff_final = keras.layers.add([ff_layer1, ff_layer2])
    ff_final = keras.layers.BatchNormalization()(ff_final)
    ff_final = keras.layers.Activation('relu')(ff_final)
    
    classifier = keras.layers.SeparableConv2D(128, (3, 3), padding='same', strides = (1, 1), name = 'DSConv1_classifier')(ff_final)
    classifier = keras.layers.BatchNormalization()(classifier)
    classifier = keras.layers.Activation('relu')(classifier)

    classifier = keras.layers.SeparableConv2D(128, (3, 3), padding='same', strides = (1, 1), name = 'DSConv2_classifier')(classifier)
    classifier = keras.layers.BatchNormalization()(classifier)
    classifier = keras.layers.Activation('relu')(classifier)

    classifier = conv_block(classifier, 'conv', 19, (1, 1), strides=(1, 1), padding='same', relu=True)

    classifier = keras.layers.Dropout(0.3)(classifier)

    classifier = keras.layers.UpSampling2D((8, 8))(classifier)
    classifier = keras.layers.Softmax(classifier)
    
    fast_scnn = keras.Model(inputs = input_layer , outputs = classifier, name = 'Fast_SCNN')
    
    fast_scnn = keras.Model(inputs = input_layer , outputs = classifier, name = 'Fast_SCNN')
    
    optimizer = keras.optimizers.SGD(momentum=0.9, lr=0.045)
    fast_scnn.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])


In [7]:
model = create_model()

W0527 14:24:28.615122 140428666545472 deprecation.py:237] From /usr/local/lib64/python3.6/site-packages/keras/backend/tensorflow_backend.py:4139: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0527 14:24:28.701297 140428666545472 deprecation.py:506] From /usr/local/lib/python3.6/site-packages/tensorflow/python/training/moving_averages.py:211: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 2))`
  


ValueError: Negative dimension size caused by subtracting 3 from 2 for 'conv2d_20/convolution' (op: 'Conv2D') with input shapes: [?,2,2,128], [3,2,128,128].

In [ ]:
(1, 2) == (1, 2)